In [1]:
print("Author:  Liubov Soldatenko")

Author:  Liubov Soldatenko


In [2]:
import os
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


import re
from collections import Counter

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


# download NLTK stop words
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liubov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/liubov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/liubov/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# <span style='color:#4682B4'> About Dataset  </span>


This dataset includes information on food choices, nutrition, preferences, childhood favorites, and other information from college students. 

There are 125 responses from students. Data is raw and uncleaned. 

The column descriptions are stored at the <a href='https://www.kaggle.com/datasets/borapajo/food-choices'>link.</a>


# <span style='color:#4682B4'> Get the Data  </span>


In [3]:
api = KaggleApi()
api.authenticate()

api.dataset_download_files('borapajo/food-choices', unzip=True)
warnings.filterwarnings('ignore')

In [4]:
#view the list of unzipped files
os.listdir()

['~$lemmatized_words.xlsx',
 '~$missing_data.xlsx',
 '.DS_Store',
 'Food choices dataset. Data Cleaning.ipynb',
 'codebook_food.docx',
 '.ipynb_checkpoints',
 'food_coded.csv']

In [5]:
df=pd.read_csv('food_coded.csv')
df.head()

,GPA,Gender,breakfast,calories_chicken,calories_day,calories_scone,coffee,comfort_food,comfort_food_reasons,comfort_food_reasons_coded,...,soup,sports,thai_food,tortilla_calories,turkey_calories,type_sports,veggies_day,vitamins,waffle_calories,weight
0,2.4,2,1,430,NaN,315.0,1,none,we dont have comfort,9.0,...,1.0,1.0,1,1165.0,345,car racing,5,1,1315,187
1,3.654,1,1,610,3.0,420.0,2,"chocolate, chips, ice cream","Stress, bored, anger",1.0,...,1.0,1.0,2,725.0,690,Basketball,4,2,900,155
2,3.3,1,1,720,4.0,420.0,2,"frozen yogurt, pizza, fast food","stress, sadness",1.0,...,1.0,2.0,5,1165.0,500,none,5,1,900,I'm not answering this.
3,3.2,1,1,430,3.0,420.0,2,"Pizza, Mac and cheese, ice cream",Boredom,2.0,...,1.0,2.0,5,725.0,690,NaN,3,1,1315,"Not sure, 240"
4,3.5,1,1,720,2.0,420.0,2,"Ice cream, chocolate, chips","Stress, boredom, cravings",1.0,...,1.0,1.0,4,940.0,500,Softball,4,2,760,190


In [6]:
print(df.shape[0], "rows in data")
print(len(df.columns), "columns in data")

125 rows in data
61 columns in data


In [7]:
#data_raw.info()
#list(data_raw.columns)
#data_raw[['comfort_food_reasons','comfort_food_reasons_coded','comfort_food_reasons_coded.1']][data_raw['comfort_food_reasons_coded.1']!=data_raw['comfort_food_reasons_coded']]

# <span style='color:#4682B4'> Remove redundant or irrelevant columns</span>


In [8]:
# <span style='color:#4682B4'> Get the Data  </span>
drop_columns_list=[
 # not usefull as we don't have reference data
 'calories_chicken',
 'calories_scone',
 'tortilla_calories',
 'turkey_calories',
 'waffle_calories',   
 'coffee',
 'breakfast',
 'drink',
 'fries',
 'soup',       
# require recodding or old coding   
 'comfort_food_reasons_coded', #only 1st reason coded, recoding needed 
 'comfort_food_reasons_coded.1',#only 1st reason coded, recoding needed  
 'diet_current', #we have coded data
 'eating_changes',#we have codded data
 'ideal_diet',  #require codding
 'eating_changes_coded', #we have eating_changes_coded1
# nor related to the students current eating prefences
 'father_education',
 'father_profession',
 'mother_education',
 'mother_profession',
 'parents_cook',    
# redundancy of data - the same data stored in fav_cuisine_coded  
 'indian_food',
 'italian_food', 
 'greek_food',
 'persian_food', 
 'thai_food',
#others    
 'healthy_meal',
 'meals_dinner_friend',
 'pay_meal_out',
 'veggies_day',
 'food_childhood']


In [9]:
data_raw=df.drop(columns=drop_columns_list).copy()
print(data_raw.shape[0], "rows in data")
print(len(data_raw.columns), "columns in data")

125 rows in data
30 columns in data


In [10]:
# lets check the list of object columns
object_columns = data_raw.select_dtypes(include=['object']).columns.tolist()
object_columns

['GPA',
 'comfort_food',
 'comfort_food_reasons',
 'fav_cuisine',
 'type_sports',
 'weight']

<b>GPA</b> and <b> weight </b>fields should exclusively contain numerical data. Therefore, it's essential to perform additional checks to detect any errors or inconsistencies in the values stored within these fields.

# <span style='color:#4682B4'> Missing values</span>


Lets start with finding missing values and filling them with data.

In [11]:
missing_data=data_raw.isna().sum().to_frame()
missing_data[missing_data[0]>0]
#missing_data.to_excel('missing_data.xlsx')

,0
GPA,2
calories_day,19
comfort_food,1
comfort_food_reasons,2
cook,3
cuisine,17
employment,9
exercise,13
fav_cuisine,2
fav_food,2


# <span style='color:#4682B4'> GPA</span>


In [12]:
print(data_raw['GPA'].dtype)# <span style='color:#4682B4'> Missing values</span>

object


It should be a numerical field. Let's check the unique values in that column.

In [13]:
data_raw['GPA'].value_counts()

GPA
3.5           13
3             11
3.2           10
3.7           10
3.3            9
3.4            9
3.6            7
3.9            7
3.8            6
2.8            5
4              4
3.1            3
2.9            2
3.83           2
2.6            2
2.4            1
3.79 bitch     1
3.73           1
2.71           1
3.92           1
3.68           1
3.75           1
Unknown        1
3.77           1
3.63           1
3.67           1
3.89           1
Personal       1
3.35           1
3.292          1
3.605          1
3.654          1
3.65           1
3.87           1
2.2            1
3.904          1
2.25           1
3.882          1
Name: count, dtype: int64

The field GPA containes sting variables:
 - 3.79 bitch (should be replaced by 3.79)
 - Unknown and Personal (as also 2 missing values could be replaced with average GPA)

In [14]:
data_raw['GPA_corrected']=data_raw['GPA']


data_raw['GPA_corrected'][data_raw['GPA'] == '3.79 bitch'] =3.79
data_raw['GPA_corrected'][data_raw['GPA'] == 'Unknown'] =np.nan
data_raw['GPA_corrected'][data_raw['GPA'] == 'Personal '] =np.nan

In [15]:
# replace all NAN values with mean GPA
data_raw['GPA_corrected'] = data_raw['GPA_corrected'].astype(float)
data_raw['GPA_corrected'].fillna(data_raw['GPA_corrected'].mean().round(3), inplace=True)

In [16]:
data_raw.drop(columns='GPA', inplace=True)

# <span style='color:#4682B4'> Weight</span>


In [17]:
print(data_raw['weight'].dtype)

object


In [18]:
data_raw['weight'].value_counts()

weight
135                         8
140                         8
150                         7
170                         7
175                         6
180                         6
155                         6
185                         6
165                         5
190                         5
125                         5
145                         4
200                         4
130                         4
120                         3
160                         3
129                         2
113                         2
128                         2
167                         2
210                         2
118                         1
192                         1
187                         1
112                         1
144 lbs                     1
127                         1
260                         1
184                         1
230                         1
138                         1
265                         1
205                         1
169

This field should be float type - we have to fix 3 values over there:
 - 144 lbs to 144
 - Not sure, 240 to 240
 - I'm not answering this to NaN and then replace all NaN with average

In [19]:
data_raw['weight_corrected']=data_raw['weight']


data_raw['weight_corrected'][data_raw['weight'] == '144 lbs'] =144
data_raw['weight_corrected'][data_raw['weight'] == 'Not sure, 240'] =240
data_raw['weight_corrected'][data_raw['weight'] == "I'm not answering this. "] =np.nan

# replace all NAN values with mean weight
data_raw['weight_corrected'] = data_raw['weight_corrected'].astype(float)
data_raw['weight_corrected'].fillna(data_raw['weight_corrected'].mean().round(0), inplace=True)

In [20]:
data_raw[['weight','weight_corrected']].head(5)

,weight,weight_corrected
0,187,187.0
1,155,155.0
2,I'm not answering this.,159.0
3,"Not sure, 240",240.0
4,190,190.0


In [21]:
data_raw['weight_corrected'].isna().any()

False

In [22]:
data_raw.drop(columns='weight', inplace=True)

# <span style='color:#4682B4'> Replaces nan-values with the most frequent value</span>


In [23]:
columns_fill_with_mode=['calories_day', 'cook','cuisine', 'employment','fav_cuisine','fav_food','income',
                        'life_rewarding','marital_status','on_off_campus','self_perception_weight']
columns_fill_with_mode

['calories_day',
 'cook',
 'cuisine',
 'employment',
 'fav_cuisine',
 'fav_food',
 'income',
 'life_rewarding',
 'marital_status',
 'on_off_campus',
 'self_perception_weight']

In [24]:
data_raw[columns_fill_with_mode].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   calories_day            106 non-null    float64
 1   cook                    122 non-null    float64
 2   cuisine                 108 non-null    float64
 3   employment              116 non-null    float64
 4   fav_cuisine             123 non-null    object 
 5   fav_food                123 non-null    float64
 6   income                  124 non-null    float64
 7   life_rewarding          124 non-null    float64
 8   marital_status          124 non-null    float64
 9   on_off_campus           124 non-null    float64
 10  self_perception_weight  124 non-null    float64
dtypes: float64(10), object(1)
memory usage: 10.9+ KB


In [25]:
for i in columns_fill_with_mode:
    most_frequent_value=data_raw[i].mode()[0]
    data_raw[i+'_corrected']=data_raw[i]
    data_raw[i+'_corrected'][data_raw[i].isna()==True]=most_frequent_value
    print("NaN values are replaces with mode for: ",i)

NaN values are replaces with mode for:  calories_day
NaN values are replaces with mode for:  cook
NaN values are replaces with mode for:  cuisine
NaN values are replaces with mode for:  employment
NaN values are replaces with mode for:  fav_cuisine
NaN values are replaces with mode for:  fav_food
NaN values are replaces with mode for:  income
NaN values are replaces with mode for:  life_rewarding
NaN values are replaces with mode for:  marital_status
NaN values are replaces with mode for:  on_off_campus
NaN values are replaces with mode for:  self_perception_weight


In [26]:
data_raw.drop(columns=columns_fill_with_mode, inplace=True)
data_raw

,Gender,comfort_food,comfort_food_reasons,diet_current_coded,eating_changes_coded1,eating_out,ethnic_food,exercise,fav_cuisine_coded,fruit_day,...,cook_corrected,cuisine_corrected,employment_corrected,fav_cuisine_corrected,fav_food_corrected,income_corrected,life_rewarding_corrected,marital_status_corrected,on_off_campus_corrected,self_perception_weight_corrected
0,2,none,we dont have comfort,1,1,3,1,1.0,3,5,...,2.0,1.0,3.0,Arabic cuisine,1.0,5.0,1.0,1.0,1.0,3.0
1,1,"chocolate, chips, ice cream","Stress, bored, anger",2,2,2,4,1.0,1,4,...,3.0,1.0,2.0,Italian,1.0,4.0,1.0,2.0,1.0,3.0
2,1,"frozen yogurt, pizza, fast food","stress, sadness",3,3,2,5,2.0,1,5,...,1.0,3.0,3.0,italian,3.0,6.0,7.0,2.0,2.0,6.0
3,1,"Pizza, Mac and cheese, ice cream",Boredom,2,3,2,5,3.0,3,4,...,2.0,2.0,3.0,Turkish,1.0,6.0,2.0,2.0,1.0,5.0
4,1,"Ice cream, chocolate, chips","Stress, boredom, cravings",2,4,2,4,1.0,1,4,...,1.0,2.0,2.0,Italian,3.0,6.0,1.0,1.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,1,"wine. mac and cheese, pizza, ice cream",boredom and sadness,2,3,2,4,2.0,1,5,...,3.0,1.0,1.0,Italian,1.0,4.0,7.0,1.0,3.0,4.0
121,1,Pizza / Wings / Cheesecake,Loneliness / Homesick / Sadness,2,3,4,3,2.0,2,4,...,3.0,1.0,3.0,Mexican Food,1.0,2.0,7.0,1.0,1.0,4.0
122,1,"rice, potato, seaweed soup",sadness,2,3,3,5,2.0,4,4,...,3.0,1.0,3.0,Korean,1.0,2.0,10.0,1.0,1.0,4.0
123,2,"Mac n Cheese, Lasagna, Pizza","happiness, they are some of my favorite foods",1,8,5,2,1.0,1,5,...,3.0,1.0,2.0,Italian,3.0,4.0,1.0,1.0,1.0,2.0


# <span style='color:#4682B4'> Comfort_food_reasons (recoding)</span>


In [27]:
# function that calculates the frequency of words in text data stored in a dataframe.
def category_frequency(df,column):
    df[column]= df[column].astype(str)
    words_raw = ' '.join(df[column]).split()
    # remove non-alphabetic characters from each item in the list
    cleaned_words = [re.sub(r'[^a-zA-Z]', '', word) for word in words_raw]

    # remove empty strings from the list
    cleaned_words = list(filter(None, cleaned_words))

    # lowercase all words
    lowercase_words = [word.lower() for word in cleaned_words]
    
    # get English stop words from NLTK
    stop_words = set(stopwords.words('english'))

    # Remove stop words from the list
    filtered_words = [word for word in lowercase_words if word.lower() not in stop_words]
    
    # lemmatizer the words
    lemmatizer = WordNetLemmatizer()

    # Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # calculate the frequency of words in a list 
    word_frequency = Counter(lemmatized_words)
    df_words_frequency = pd.DataFrame.from_dict(word_frequency, orient='index', columns=['frequency'])
    df_words_frequency.sort_values(by='frequency', ascending=False, inplace=True)
    
    return df_words_frequency

    

In [28]:
df_food_reasons=data_raw['comfort_food_reasons'].copy().to_frame()

In [29]:
comfort_food_reasons_frequency=category_frequency(df_food_reasons, 'comfort_food_reasons')
comfort_food_reasons_frequency.head(10)

,frequency
boredom,74
sadness,42
stress,33
anger,10
comfort,10
happiness,9
bored,9
food,9
eat,7
sad,7


In [30]:
#based on frequency we have created the list of categories
comfort_food_reasons_categories=['boredom', 'sadness','stress','hunger','anger', 'happy','other','none']


df_food_reasons['comfort_food_reasons'] = df_food_reasons['comfort_food_reasons'].str.lower()

for i in comfort_food_reasons_categories:
    df_food_reasons['cfr_'+i]=0


df_food_reasons.head()

,comfort_food_reasons,cfr_boredom,cfr_sadness,cfr_stress,cfr_hunger,cfr_anger,cfr_happy,cfr_other,cfr_none
0,we dont have comfort,0,0,0,0,0,0,0,0
1,"stress, bored, anger",0,0,0,0,0,0,0,0
2,"stress, sadness",0,0,0,0,0,0,0,0
3,boredom,0,0,0,0,0,0,0,0
4,"stress, boredom, cravings",0,0,0,0,0,0,0,0


In [31]:
df_cfr=df_food_reasons.copy()

In [32]:
# 'boredom' and 'bored' should be catagorized as 'boredom'
df_cfr['cfr_boredom'][df_cfr['comfort_food_reasons'].str.contains(r"bored")==True]=1

# 'stressed' and 'stress' should be catagorized as 'stress'
df_cfr['cfr_stress'][df_cfr['comfort_food_reasons'].str.contains(r"stress")==True]=1

# 'sad' and 'sadness' and 'depression' should be catagorized as 'sadness'
df_cfr['cfr_sadness'][(df_cfr['comfort_food_reasons'].str.contains(r"sad")==True) |
                      (df_cfr['comfort_food_reasons'].str.contains(r"depress")==True)]=1

# 'hunger' and 'hungry' should be catagorized as 'hunger' 
df_cfr['cfr_hunger'][(df_cfr['comfort_food_reasons'].str.contains(r"hunger")==True) |
                                      (df_cfr['comfort_food_reasons'].str.contains(r"hungry")==True)]=1

# 'happy' and 'happiness' should be catagorized as 'happy'
df_cfr['cfr_happy'][(df_cfr['comfort_food_reasons'].str.contains(r"happ")==True)]=1

# 'anger' should be catagorized as 'anger'
df_cfr['cfr_anger'][(df_cfr['comfort_food_reasons'].str.contains(r"ang")==True)]=1


# 'none', 'nan','naan' and NaN should be catagorized as 'none'
df_cfr['cfr_none'][(df_cfr['comfort_food_reasons'].str.contains('^nan$', na=False)==True) |
                                      (df_cfr['comfort_food_reasons'].str.contains('^nan$', na=False)==True)|
                                      (df_cfr['comfort_food_reasons'].str.contains("no reasons")==True)|
                                      (df_cfr['comfort_food_reasons'].isna()==True)]=1

# 'others' for other reasons


df_cfr.head()

,comfort_food_reasons,cfr_boredom,cfr_sadness,cfr_stress,cfr_hunger,cfr_anger,cfr_happy,cfr_other,cfr_none
0,we dont have comfort,0,0,0,0,0,0,0,0
1,"stress, bored, anger",1,0,1,0,1,0,0,0
2,"stress, sadness",0,1,1,0,0,0,0,0
3,boredom,1,0,0,0,0,0,0,0
4,"stress, boredom, cravings",1,0,1,0,0,0,0,0


In [33]:
# other 
df_cfr['sum']=0
total_sum=0
for i in comfort_food_reasons_categories:
    total_sum +=df_cfr['cfr_'+i]
    df_cfr['sum']=total_sum

    
df_cfr['cfr_other'][df_cfr['sum']==0]=1
df_cfr.drop(columns=['sum'], inplace=True)

In [34]:
df_cfr[df_cfr['cfr_other']==1].head(20)

,comfort_food_reasons,cfr_boredom,cfr_sadness,cfr_stress,cfr_hunger,cfr_anger,cfr_happy,cfr_other,cfr_none
0,we dont have comfort,0,0,0,0,0,0,1,0
24,"a long day, not feeling well, winter",0,0,0,0,0,0,1,0
41,tired,0,0,0,0,0,0,1,0
49,they taste better than other food. they are a ...,0,0,0,0,0,0,1,0
51,lazy,0,0,0,0,0,0,1,0
57,just cause,0,0,0,0,0,0,1,0
94,laziness and hungover,0,0,0,0,0,0,1,0
103,"anxiousness, watching tv i desire ""comfort food""",0,0,0,0,0,0,1,0
117,when i'm eating with my close friends/ food s...,0,0,0,0,0,0,1,0
124,"hormones, premenstrual syndrome.",0,0,0,0,0,0,1,0


# <span style='color:#4682B4'> Comfort_food (coding)</span>


In [35]:
df_comfort_food=data_raw['comfort_food'].copy().to_frame()
comfort_food_frequency=category_frequency(df_comfort_food, 'comfort_food')
comfort_food_frequency.head(15)

,frequency
ice,50
cream,47
pizza,40
chocolate,36
chip,34
cheese,22
cooky,19
mac,17
chicken,14
fry,11


In [36]:
#based on frequency we have created the list of categories
comfort_food_categories=['ice_cream', 'sweets', 'pizza','chips', 'pasta','burger','french_fries','other','none']


df_comfort_food['comfort_food'] = df_comfort_food['comfort_food'].str.lower()

for i in comfort_food_categories:
    df_comfort_food['cf_'+i]=0


df_comfort_food.head()

,comfort_food,cf_ice_cream,cf_sweets,cf_pizza,cf_chips,cf_pasta,cf_burger,cf_french_fries,cf_other,cf_none
0,none,0,0,0,0,0,0,0,0,0
1,"chocolate, chips, ice cream",0,0,0,0,0,0,0,0,0
2,"frozen yogurt, pizza, fast food",0,0,0,0,0,0,0,0,0
3,"pizza, mac and cheese, ice cream",0,0,0,0,0,0,0,0,0
4,"ice cream, chocolate, chips",0,0,0,0,0,0,0,0,0


In [37]:
# ice cream
df_comfort_food['cf_ice_cream'][df_comfort_food['comfort_food'].str.contains(r"ice")==True]=1

# pizza
df_comfort_food['cf_pizza'][df_comfort_food['comfort_food'].str.contains(r"pizza")==True]=1

# sweets: chocolate, cookies, candies, brownies
df_comfort_food['cf_sweets'][(df_comfort_food['comfort_food'].str.contains(r"chocolat")==True)|
                               (df_comfort_food['comfort_food'].str.contains(r"cookies")==True)|
                               (df_comfort_food['comfort_food'].str.contains(r"cand")==True)|
                               (df_comfort_food['comfort_food'].str.contains(r"brown")==True)]=1
                  
# chips
df_comfort_food['cf_chips'][df_comfort_food['comfort_food'].str.contains(r"chip")==True]=1

# pasta: pasta and mac&cheese
df_comfort_food['cf_pasta'][(df_comfort_food['comfort_food'].str.contains(r"pasta")==True) |
                           (df_comfort_food['comfort_food'].str.contains("mac")==True)]=1

# burger
df_comfort_food['cf_burger'][df_comfort_food['comfort_food'].str.contains("burger")==True]=1

# french fries
df_comfort_food['cf_french_fries'][df_comfort_food['comfort_food'].str.contains("french fries")==True]=1

# 'none', 'nan','naan' and NaN should be catagorized as 'none'
df_comfort_food['cf_none'][(df_comfort_food['comfort_food'].str.contains('^none$', na=False)==True) |
                                      (df_comfort_food['comfort_food'].str.contains('^nan$', na=False)==True)|
                                      (df_comfort_food['comfort_food'].isna()==True)]=1

df_comfort_food[df_comfort_food['cf_sweets']==1].head(20)

,comfort_food,cf_ice_cream,cf_sweets,cf_pizza,cf_chips,cf_pasta,cf_burger,cf_french_fries,cf_other,cf_none
1,"chocolate, chips, ice cream",1,1,0,1,0,0,0,0,0
4,"ice cream, chocolate, chips",1,1,0,1,0,0,0,0,0
5,"candy, brownies and soda.",0,1,0,0,0,0,0,0,0
6,"chocolate, ice cream, french fries, pretzels",1,1,0,0,0,0,1,0,0
9,"mac and cheese, chocolate, and pasta",0,1,0,0,1,0,0,0,0
10,"pasta, grandma homemade chocolate cake anythin...",0,1,0,0,1,0,0,0,0
11,"chocolate, pasta, soup, chips, popcorn",0,1,0,1,1,0,0,0,0
12,"cookies, popcorn, and chips",0,1,0,1,0,0,0,0,0
13,"ice cream, cake, chocolate",1,1,0,0,0,0,0,0,0
15,"cookies, donuts, candy bars",0,1,0,0,0,0,0,0,0


In [38]:
# other 
df_comfort_food['sum']=0
total_sum=0
for i in comfort_food_categories:
    total_sum +=df_comfort_food['cf_'+i]
    df_comfort_food['sum']=total_sum

    
df_comfort_food['cf_other'][df_comfort_food['sum']==0]=1


df_comfort_food.drop(columns=['sum'], inplace=True)

In [39]:
df_comfort_food[df_comfort_food['cf_none']==1].head()

,comfort_food,cf_ice_cream,cf_sweets,cf_pizza,cf_chips,cf_pasta,cf_burger,cf_french_fries,cf_other,cf_none
0,none,0,0,0,0,0,0,0,0,1
74,nan,0,0,0,0,0,0,0,0,1


# <span style='color:#4682B4'> Sport, Type Sport, Exercise</span>


As these fields are connected, we will fix the data in them together. <br>
We need to check:
 - If the student doesn't participate in any sporting activity (sports=2), then the type_sports should be 'none'. And 'excersize' should be 5 (which means 'Never')

 - if the student fill the field type_sports, then sports should be '1'

In [40]:
df_sport=data_raw[['sports','type_sports','exercise']].copy()

In [41]:
df_sport[df_sport['sports'].isna()]['type_sports']

19      basketball
121    basketball 
Name: type_sports, dtype: object

Two students mentioned that they are playing basketball, but the value of field 'sports' in empty - we will replace it with '1'.

In [42]:
df_sport['sports']=df_sport['sports'].fillna(value=1)
df_sport['sports'].isna().any()

False

In [43]:
# lets check 'exercise' field
print (df_sport['exercise'].isna().sum(), "total NaN values in 'exercise' field")
print(df_sport[df_sport['sports']==2]['exercise'].isna().sum(), " NaN values in the dataset corresponding to students who don't participate in sports. These values should be corrected to '5' (which represents 'Never')")
print(df_sport[df_sport['sports']==1]['exercise'].isna().sum()," NaN values in the dataset corresponding to students who do sports. These values we will replace by the most frequent 'excercise' value")

13 total NaN values in 'exercise' field
8  NaN values in the dataset corresponding to students who don't participate in sports. These values should be corrected to '5' (which represents 'Never')
5  NaN values in the dataset corresponding to students who do sports. These values we will replace by the most frequent 'excercise' value


In [44]:
df_sport['exercise_corrected']=df_sport['exercise']
df_sport['exercise_corrected'][(df_sport['sports'] == 2) & (df_sport['exercise'].isna()==True)] =5
df_sport['exercise_corrected'][(df_sport['sports'] == 1) & (df_sport['exercise'].isna()==True)] =df_sport['exercise'].mode()[0]

print(df_sport[df_sport['exercise_corrected']!=df_sport['exercise']].shape[0],"NaN values in field 'exercise' were fixed ")


13 NaN values in field 'exercise' were fixed 


Before fixing NaN values in the 'type_sports' field, we need to clean the data stored in this field. <br>Let's check the unique values for all students who mentioned they don't do any sports activity:

In [45]:
list(df_sport['type_sports'][df_sport['sports']==2].unique())

['none',
 nan,
 'None.',
 'dancing ',
 'none organized',
 'no particular engagement ',
 'I danced in high school',
 'None right now',
 'none ',
 'crew',
 'Soccer',
 'When I can, rarely though play pool, darts, and basketball.',
 'None at the moment',
 'I used to play softball ',
 ' None',
 "No, I don't play sport."]

In [46]:
# for all students that mentioned they don't do sports we should assign value 'No Sport' for field type_sports
df_sport.loc[df_sport['sports'] == 2, 'type_sports'] ='No Sport'
list(df_sport['type_sports'][df_sport['sports']==2].unique())

['No Sport']

In [47]:
df_sport[df_sport['sports']==2]['type_sports'].isna().sum()

0

The nan values where fixed automatically to "No Sport". <br>
Now we will clean other values in the field 'type_sports': check grammatic, group the sports activities and code tht most frequent of them.

In [48]:
# lets identify the most frequent sports
sport_frequency=category_frequency(df_sport, 'type_sports')
sport_frequency.head(15)

,frequency
sport,48
hockey,15
soccer,10
basketball,9
softball,9
volleyball,9
tennis,5
wrestling,5
lacrosse,5
running,4


In [49]:
# based on the frequency of sports, we will create a sport category list
sport_category=['hockey','soccer','basketball','softball','volleyball','tennis',
                'lacrosse','wrestling','running','skiing', 'no sport'] # for the rest not frequent sport types

df_sport['type_sports'] = df_sport['type_sports'].str.lower()

In [50]:
# splitting sport data into categories
for i in sport_category:
    df_sport.loc[df_sport['type_sports'].str.contains(i) == True, i]=1
    df_sport[i].fillna(value=0, inplace=True)
    df_sport[i] = df_sport[i].astype(int)

In [51]:
# add all sport activities that are not included in sport_category list to 'other sport' category
total_sum=0
for i in sport_category:
    total_sum +=df_sport[i]
    df_sport['sum']=total_sum
    
df_sport.loc[df_sport['sum'] == 0, 'other sport'] =1
df_sport.loc[df_sport['sum'] > 0, 'other sport'] =0


In [52]:
df_sport.drop(columns=['type_sports','sum','exercise','no sport'], inplace=True)
df_sport = df_sport.astype(int) 
df_sport.head(10)

,sports,exercise_corrected,hockey,soccer,basketball,softball,volleyball,tennis,lacrosse,wrestling,running,skiing,other sport
0,1,1,0,0,0,0,0,0,0,0,0,0,1
1,1,1,0,0,1,0,0,0,0,0,0,0,0
2,2,2,0,0,0,0,0,0,0,0,0,0,0
3,2,3,0,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,0,0,0
5,2,2,0,0,0,0,0,0,0,0,0,0,0
6,1,1,0,1,0,0,0,0,0,0,0,0,0
7,2,2,0,0,0,0,0,0,0,0,0,0,0
8,2,5,0,0,0,0,0,0,0,0,0,0,0
9,1,1,1,0,0,0,0,0,0,0,0,0,0


In [53]:
df_sport.isna().any()

sports                False
exercise_corrected    False
hockey                False
soccer                False
basketball            False
softball              False
volleyball            False
tennis                False
lacrosse              False
wrestling             False
running               False
skiing                False
other sport           False
dtype: bool

Sport data is cleaned and split into categories, therefore it can be used for further analysis.

# <span style='color:#4682B4'> Create a clean dataset, right eady for analysis</span>


In [54]:
data_raw.columns

Index(['Gender', 'comfort_food', 'comfort_food_reasons', 'diet_current_coded',
       'eating_changes_coded1', 'eating_out', 'ethnic_food', 'exercise',
       'fav_cuisine_coded', 'fruit_day', 'grade_level', 'healthy_feeling',
       'ideal_diet_coded', 'nutritional_check', 'sports', 'type_sports',
       'vitamins', 'GPA_corrected', 'weight_corrected',
       'calories_day_corrected', 'cook_corrected', 'cuisine_corrected',
       'employment_corrected', 'fav_cuisine_corrected', 'fav_food_corrected',
       'income_corrected', 'life_rewarding_corrected',
       'marital_status_corrected', 'on_off_campus_corrected',
       'self_perception_weight_corrected'],
      dtype='object')

In [55]:
data_raw.shape

(125, 30)

In [56]:
columns_with_missing_values = data_raw.columns[data_raw.isna().any()].tolist()
columns_with_missing_values

['comfort_food', 'comfort_food_reasons', 'exercise', 'sports', 'type_sports']

In [57]:
data_raw.drop(columns=columns_with_missing_values, inplace=True)

In [58]:
# concat with correct data
data_clean = pd.concat([data_raw, df_sport, df_comfort_food,df_cfr], axis=1)
data_clean.head()

,Gender,diet_current_coded,eating_changes_coded1,eating_out,ethnic_food,fav_cuisine_coded,fruit_day,grade_level,healthy_feeling,ideal_diet_coded,...,cf_none,comfort_food_reasons,cfr_boredom,cfr_sadness,cfr_stress,cfr_hunger,cfr_anger,cfr_happy,cfr_other,cfr_none
0,2,1,1,3,1,3,5,2,2,8,...,1,we dont have comfort,0,0,0,0,0,0,1,0
1,1,2,2,2,4,1,4,4,5,3,...,0,"stress, bored, anger",1,0,1,0,1,0,0,0
2,1,3,3,2,5,1,5,3,6,6,...,0,"stress, sadness",0,1,1,0,0,0,0,0
3,1,2,3,2,5,3,4,4,7,2,...,0,boredom,1,0,0,0,0,0,0,0
4,1,2,4,2,4,1,4,4,6,2,...,0,"stress, boredom, cravings",1,0,1,0,0,0,0,0


In [59]:
data_clean.columns[data_clean.isna().any()].tolist()

[]

In [60]:
data_clean.select_dtypes(include=['object']).columns.tolist()

['fav_cuisine_corrected', 'comfort_food', 'comfort_food_reasons']

In [61]:
data_clean.shape

(125, 57)

<b> data_clean </b>: 

1) Contains 57 columns.
2) NaN are fixed.
3) Data types are fixed.
4) All text columns were coded using NLP methods.